In [7]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import numpy as np
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import gensim
from gensim import models
from gensim import corpora
import lda

In [4]:
stop_words = ['rt','_']
stop_words

['rt', '_']

In [ ]:
def preprocessing(row):
    stopword_removed_output = [w for w in row.split(' ') if not w in stop_words]
    return stopword_removed_output

In [10]:
train_input = pd.read_csv('s3://recsys-challenge-2020/train_input.csv')
val_input = pd.read_csv('s3://recsys-challenge-2020/val_input.csv')
test_input = pd.read_csv('s3://recsys-challenge-2020/test_input.csv')

In [ ]:
all_data = pd.concat(train_input, val_input, test_input)

In [13]:
full_input = all_input.sort_values(by='name')

In [ ]:
languages = list(full_input.name.unique())

In [ ]:
all_lang_output = []
for language in languages:
    lang_input = full_input.loc[full_input.name == language]
    lang_input['tweet_tokens'] = lang_input.apply(lambda x: preprocessing(x.tweet_text), axis = 1)
    dictionary = corpora.Dictionary(lang_input.tweet_tokens)
    corpus = [dictionary.doc2bow(text) for text in lang_input.tweet_tokens]
    ldamodel = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = 100, id2word=dictionary, passes=5, workers=96)
    all_topics = ldamodel.get_document_topics(corpus, minimum_probability=0.0)
    all_topics_csr = gensim.matutils.corpus2csc(all_topics)
    all_topics_numpy = all_topics_csr.T.toarray()
    all_topics_df = pd.DataFrame(all_topics_numpy)
    all_topics_df.index = lang_input.index
    lang_output = pd.concat([lang_input, all_topics_df], axis = 1)
    all_lang_output = all_lang_output.append(lang_output)
all_lang_output = pd.concat(all_lang_output)